# SLEAP Distance Calculation

Brief 1-2 sentence description of notebook.

In [1]:
import os
import glob
import git
import sys


In [2]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
# import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import h5py
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter


In [3]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [4]:
git_root

'/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/reward_competition_extention'

In [5]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [6]:
import utilities.helper
import sleap.process_pose

In [7]:
# sns.set('notebook', 'ticks', font_scale=1.2)
mpl.rcParams['figure.figsize'] = [15,6]

# Functions

In [8]:
def compute_velocity(node_loc, window_size=25, polynomial_order=3):
    """
    Calculate the velocity of tracked nodes from pose data.
    
    The function utilizes the Savitzky-Golay filter to smooth the data and compute the velocity.
    
    Parameters:
    ----------
    node_loc : numpy.ndarray
        The location of nodes, represented as an array of shape [frames, 2]. 
        Each row represents x and y coordinates for a particular frame.
        
    window_size : int, optional
        The size of the window used for the Savitzky-Golay filter. 
        Represents the number of consecutive data points used when smoothing the data.
        Default is 25.
        
    polynomial_order : int, optional
        The order of the polynomial fit to the data within the Savitzky-Golay filter window.
        Default is 3.

    Returns:
    -------
    numpy.ndarray
        The velocity for each frame, calculated from the smoothed x and y coordinates.
    
    """
    node_loc_vel = np.zeros_like(node_loc)
    
    # For each coordinate (x and y), smooth the data and calculate the derivative (velocity)
    for c in range(node_loc.shape[-1]):
        node_loc_vel[:, c] = savgol_filter(node_loc[:, c], window_size, polynomial_order, deriv=1)
    
    # Calculate the magnitude of the velocity vectors for each frame
    node_vel = np.linalg.norm(node_loc_vel, axis=1)

    return node_vel

In [9]:
def rolling_average(arr, window_size):
    """
    Computes the rolling average using a specified window size.
    
    Parameters:
        arr (numpy.array): The input array to compute the rolling average for.
        window_size (int): The size of the rolling window.

    Returns:
        numpy.array: The rolling average of the input array.
    """
    if window_size < 1:
       raise ValueError("Window size must be at least 1.")
    
    # Create a uniform window of given window size
    window = np.ones(window_size) / window_size

    # Use numpy's convolve function to compute the rolling average
    return np.convolve(arr, window, mode='valid')



In [10]:
def chunked_average(arr, chunk_size):
    """
    Computes the average for non-overlapping chunks of the input array.
    
    Parameters:
        arr (numpy.array): The input array.
        chunk_size (int): The size of each chunk.

    Returns:
        numpy.array: The averages of the non-overlapping chunks.
    """

    # Number of chunks
    num_chunks = len(arr) // chunk_size
    
    # Reshape the array into a 2D array of shape (num_chunks, chunk_size)
    reshaped_arr = arr[:num_chunks * chunk_size].reshape(num_chunks, chunk_size)
    
    # Compute the mean along the second axis (i.e., for each chunk)
    return reshaped_arr.mean(axis=1)

In [11]:
def sliding_window_average(arr, window_size, step=1):
    """
    Apply a sliding window to a 1D numpy array, returning the average of windows of a specified size.

    :param arr: Input 1D numpy array.
    :param window_size: Size of the window.
    :param step: The step size or number of elements to slide the window by. Default is 1.
    :return: A 1D numpy array where each element is the average of a window from the input.
    """
    # Number of windows
    num_windows = ((arr.size - window_size) // step) + 1
    
    # Output array for averages
    averages = np.zeros(num_windows)
    
    for i in range(num_windows):
        # Calculate the start and end index for the window
        start = i * step
        end = start + window_size
        # Calculate the average of the window
        averages[i] = np.mean(arr[start:end])

    return averages

In [12]:
def calculate_all_window_indices(original_index, window_size, step, array_length):
    """
    Calculate all the start and stop indices for sliding windows based on an original start index.

    :param original_index: The original index from which the first window should start.
    :param window_size: The size of each sliding window.
    :param step: The step size or number of elements to slide the window by.
    :param array_length: The total number of elements in the array.
    :return: A list of tuples, each containing the start and stop indices for a sliding window.
    """

    # Initialize the list to hold the start and stop indices for all windows
    windows = []

    # Initialize the current start index with the original index
    current_start_index = original_index

    # Loop through the array until the end is reached
    while current_start_index + window_size <= original_index + array_length:
        # Calculate the stop index based on the window size
        stop_index = current_start_index + window_size

        # Add the start and stop indices to the list
        windows.append((current_start_index, stop_index))

        # Update the current start index by adding the step size
        current_start_index += step

    return windows

## Inputs & Data

Explanation of each input and where it comes from.

In [14]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
THORAX_INDEX = 1

# VIDEO_TO_FRAME_AND_SUBJECT_DF["video_name"] = VIDEO_TO_FRAME_AND_SUBJECT_DF["video_name"].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))

# SLEAP_DIR = os.path.join(git_root, "proc/sleap") 
# SLEAP_DIR = "/scratch/back_up/reward_competition_extention/final_proc/id_corrected"
SLEAP_DIR = "/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/id_corrected"

OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs
MED_PC_WIDTH = 29.5
MED_PC_HEIGHT = 24
FRAME_RATE = 22
WINDOW_SIZE = 25
DISTANCE_THRESHOLD = 2

In [15]:
START_STOP_FRAME_DF = pd.read_excel("./data/rce_per_subject_start_stop_video_frame.xlsx")


## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

In [17]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_2"

In [18]:
FULL_LFP_TRACES_PKL = "{}_full_spectral_and_sleap_poses.pkl".format(OUTPUT_PREFIX)

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

# Getting the videos where the subject is in the recording

### Looking at when each subject was in each video

In [45]:
START_STOP_FRAME_DF = pd.read_excel("./data/rce_per_subject_start_stop_video_frame.xlsx")
START_STOP_FRAME_DF = START_STOP_FRAME_DF.dropna(subset=["file_path"])

- Getting the name of the SLEAP and video files where each subject was in

In [46]:
START_STOP_FRAME_DF["sleap_name"] = START_STOP_FRAME_DF["file_path"].apply(lambda x: os.path.basename(x))
START_STOP_FRAME_DF["video_name"] = START_STOP_FRAME_DF["file_path"].apply(lambda x: ".".join(os.path.basename(x).split(".")[:2]))
START_STOP_FRAME_DF["start_frame"] = START_STOP_FRAME_DF["start_frame"].astype(int)
START_STOP_FRAME_DF["stop_frame"] = START_STOP_FRAME_DF["stop_frame"].astype(int)

In [47]:
START_STOP_FRAME_DF = START_STOP_FRAME_DF.drop(columns=["file_path", "notes"], errors="ignore")

In [48]:
START_STOP_FRAME_DF["video_name"].unique()

array(['20221214_125409_om_and_comp_6_1_and_6_3.1',
       '20221215_145401_comp_amd_om_6_1_and_6_3.1',
       '20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1.1',
       '20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1.2',
       '20230613_105657_standard_comp_to_training_D2_subj_1-1_and_1-4.1',
       '20230613_105657_standard_comp_to_training_D2_subj_1-1_and_1-4.2',
       '20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2.1',
       '20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2.2',
       '20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2.3',
       '20230616_111904_standard_comp_to_training_D4_subj_1-4_and_1-2.1',
       '20230616_111904_standard_comp_to_training_D4_subj_1-4_and_1-2.2',
       '20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.1',
       '20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.3',
       '20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1.1',
       '

In [49]:
START_STOP_FRAME_DF.head()

,start_frame,stop_frame,tracked_subject,in_video_subjects,box_number,sleap_name,video_name
0,1,25000,6.3,6.1_6.3,1,20221214_125409_om_and_comp_6_1_and_6_3.1.fixe...,20221214_125409_om_and_comp_6_1_and_6_3.1
1,27500,73600,6.1_6.3,6.1_6.3,1,20221214_125409_om_and_comp_6_1_and_6_3.1.fixe...,20221214_125409_om_and_comp_6_1_and_6_3.1
2,51500,76454,6.3,6.1_6.3,1,20221215_145401_comp_amd_om_6_1_and_6_3.1.fixe...,20221215_145401_comp_amd_om_6_1_and_6_3.1
3,1,48500,6.1_6.3,6.1_6.3,1,20221215_145401_comp_amd_om_6_1_and_6_3.1.fixe...,20221215_145401_comp_amd_om_6_1_and_6_3.1
4,32700,68257,1.2,1.1_1.2,1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...


- Splitting each row into seperate row for each subject in the video

In [50]:
START_STOP_FRAME_DF["tracked_subject"] = START_STOP_FRAME_DF["tracked_subject"].apply(lambda x: str(x).split("_"))
START_STOP_FRAME_DF["current_subject"] = START_STOP_FRAME_DF["tracked_subject"]

In [51]:
START_STOP_FRAME_DF = START_STOP_FRAME_DF.explode("current_subject")

In [52]:
START_STOP_FRAME_DF.head()

,start_frame,stop_frame,tracked_subject,in_video_subjects,box_number,sleap_name,video_name,current_subject
0,1,25000,[6.3],6.1_6.3,1,20221214_125409_om_and_comp_6_1_and_6_3.1.fixe...,20221214_125409_om_and_comp_6_1_and_6_3.1,6.3
1,27500,73600,"[6.1, 6.3]",6.1_6.3,1,20221214_125409_om_and_comp_6_1_and_6_3.1.fixe...,20221214_125409_om_and_comp_6_1_and_6_3.1,6.1
1,27500,73600,"[6.1, 6.3]",6.1_6.3,1,20221214_125409_om_and_comp_6_1_and_6_3.1.fixe...,20221214_125409_om_and_comp_6_1_and_6_3.1,6.3
2,51500,76454,[6.3],6.1_6.3,1,20221215_145401_comp_amd_om_6_1_and_6_3.1.fixe...,20221215_145401_comp_amd_om_6_1_and_6_3.1,6.3
3,1,48500,"[6.1, 6.3]",6.1_6.3,1,20221215_145401_comp_amd_om_6_1_and_6_3.1.fixe...,20221215_145401_comp_amd_om_6_1_and_6_3.1,6.1


# Reading in the h5 files between recordings

In [53]:
START_STOP_FRAME_DF["sleap_glob"] = START_STOP_FRAME_DF["sleap_name"].apply(lambda x: glob.glob(os.path.join(SLEAP_DIR, "**", x)))


In [54]:
START_STOP_FRAME_DF["sleap_name"].iloc[16]

'20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2.1.2_subj.id_corrected.h5'

In [55]:
START_STOP_FRAME_DF = START_STOP_FRAME_DF[START_STOP_FRAME_DF['sleap_glob'].apply(lambda x: len(x) >= 1)]
START_STOP_FRAME_DF = START_STOP_FRAME_DF.reset_index(drop=True)




In [56]:
START_STOP_FRAME_DF["sleap_path"] = START_STOP_FRAME_DF["sleap_glob"].apply(lambda x: x[0])

In [57]:
START_STOP_FRAME_DF["all_sleap_data"] = START_STOP_FRAME_DF["sleap_path"].apply(lambda x: sleap.process_pose.extract_sleap_data(x))


In [58]:
START_STOP_FRAME_DF["body_parts"] = START_STOP_FRAME_DF["sleap_path"].apply(lambda x: sleap.process_pose.get_node_names_from_sleap(x))

In [59]:
START_STOP_FRAME_DF["body_parts"].iloc[0]

['left_ear', 'right_ear', 'nose', 'tail_base', 'thorax', 'forehead']

In [60]:
START_STOP_FRAME_DF["locations"] = START_STOP_FRAME_DF["all_sleap_data"].apply(lambda x: x["locations"])

In [61]:
START_STOP_FRAME_DF["track_names"] = START_STOP_FRAME_DF["all_sleap_data"].apply(lambda x: x["track_names"])

In [62]:
START_STOP_FRAME_DF["locations"].iloc[0].shape

(68258, 6, 2, 1)

In [63]:
START_STOP_FRAME_DF.head()

,start_frame,stop_frame,tracked_subject,in_video_subjects,box_number,sleap_name,video_name,current_subject,sleap_glob,sleap_path,all_sleap_data,body_parts,locations,track_names
0,32700,68257,[1.2],1.1_1.2,1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.2,[/blue/npadillacoreano/ryoi360/projects/reward...,/blue/npadillacoreano/ryoi360/projects/reward_...,{'locations': [[[[244.3555603 ] [395.80090332...,"[left_ear, right_ear, nose, tail_base, thorax,...","[[[[244.3555603], [395.80090332]], [[247.78367...",[1.2]
1,1,32300,"[1.1, 1.2]",1.1_1.2,1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,[/blue/npadillacoreano/ryoi360/projects/reward...,/blue/npadillacoreano/ryoi360/projects/reward_...,{'locations': [[[[331.99508667 244.3555603 ] ...,"[left_ear, right_ear, nose, tail_base, thorax,...","[[[[331.99508667 244.3555603 ], [127.74658203 ...","[1.1, 1.2]"
2,1,32300,"[1.1, 1.2]",1.1_1.2,1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.2,[/blue/npadillacoreano/ryoi360/projects/reward...,/blue/npadillacoreano/ryoi360/projects/reward_...,{'locations': [[[[331.99508667 244.3555603 ] ...,"[left_ear, right_ear, nose, tail_base, thorax,...","[[[[331.99508667 244.3555603 ], [127.74658203 ...","[1.1, 1.2]"
3,33000,68212,[1.1],1.1_1.2,2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,[/blue/npadillacoreano/ryoi360/projects/reward...,/blue/npadillacoreano/ryoi360/projects/reward_...,"{'locations': [[[[nan] [nan]], [[nan] [nan]]...","[left_ear, right_ear, nose, tail_base, thorax,...","[[[[nan], [nan]], [[nan], [nan]], [[nan], [nan...",[1.1]
4,33400,68332,[1.1],1.1_1.4,1,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,1.1,[/blue/npadillacoreano/ryoi360/projects/reward...,/blue/npadillacoreano/ryoi360/projects/reward_...,{'locations': [[[[336.33197021] [400.23355103...,"[left_ear, right_ear, nose, tail_base, thorax,...","[[[[336.33197021], [400.23355103]], [[319.3523...",[1.1]


In [64]:
# Getting the indexes of each subject from the track list
START_STOP_FRAME_DF["subject_to_index"] = START_STOP_FRAME_DF.apply(lambda x: {k: x["track_names"].index(k) for k in x["tracked_subject"] if k in x["track_names"]}, axis=1)

In [66]:
START_STOP_FRAME_DF["subject_to_index"].head()

0              {'1.2': 0}
1    {'1.1': 0, '1.2': 1}
2    {'1.1': 0, '1.2': 1}
3              {'1.1': 0}
4              {'1.1': 0}
Name: subject_to_index, dtype: object

In [67]:
START_STOP_FRAME_DF["subject_to_tracks"] = START_STOP_FRAME_DF.apply(lambda x: {k:v for k, v in x["subject_to_index"].items()}, axis=1)

In [69]:
START_STOP_FRAME_DF["subject_to_tracks"] = START_STOP_FRAME_DF.apply(lambda x: {k: x["locations"][:,:,:,v] for k, v in x["subject_to_index"].items()}, axis=1)

In [72]:
START_STOP_FRAME_DF["subject_to_tracks"].head()

0    {'1.2': [[[244.3555603  395.80090332], [247.78...
1    {'1.1': [[[331.99508667 127.74658203], [307.31...
2    {'1.1': [[[331.99508667 127.74658203], [307.31...
3    {'1.1': [[[nan nan], [nan nan], [nan nan], [na...
4    {'1.1': [[[336.33197021 400.23355103], [319.35...
Name: subject_to_tracks, dtype: object

In [73]:
START_STOP_FRAME_DF["subject_to_tracks"].apply(lambda x: x.keys()).head()

0         (1.2)
1    (1.1, 1.2)
2    (1.1, 1.2)
3         (1.1)
4         (1.1)
Name: subject_to_tracks, dtype: object

In [71]:
START_STOP_FRAME_DF.head()

,start_frame,stop_frame,tracked_subject,in_video_subjects,box_number,sleap_name,video_name,current_subject,sleap_glob,sleap_path,all_sleap_data,body_parts,locations,track_names,subject_to_index,subject_to_tracks
0,32700,68257,[1.2],1.1_1.2,1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.2,[/blue/npadillacoreano/ryoi360/projects/reward...,/blue/npadillacoreano/ryoi360/projects/reward_...,{'locations': [[[[244.3555603 ] [395.80090332...,"[left_ear, right_ear, nose, tail_base, thorax,...","[[[[244.3555603], [395.80090332]], [[247.78367...",[1.2],{'1.2': 0},"{'1.2': [[[244.3555603 395.80090332], [247.78..."
1,1,32300,"[1.1, 1.2]",1.1_1.2,1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,[/blue/npadillacoreano/ryoi360/projects/reward...,/blue/npadillacoreano/ryoi360/projects/reward_...,{'locations': [[[[331.99508667 244.3555603 ] ...,"[left_ear, right_ear, nose, tail_base, thorax,...","[[[[331.99508667 244.3555603 ], [127.74658203 ...","[1.1, 1.2]","{'1.1': 0, '1.2': 1}","{'1.1': [[[331.99508667 127.74658203], [307.31..."
2,1,32300,"[1.1, 1.2]",1.1_1.2,1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.2,[/blue/npadillacoreano/ryoi360/projects/reward...,/blue/npadillacoreano/ryoi360/projects/reward_...,{'locations': [[[[331.99508667 244.3555603 ] ...,"[left_ear, right_ear, nose, tail_base, thorax,...","[[[[331.99508667 244.3555603 ], [127.74658203 ...","[1.1, 1.2]","{'1.1': 0, '1.2': 1}","{'1.1': [[[331.99508667 127.74658203], [307.31..."
3,33000,68212,[1.1],1.1_1.2,2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,[/blue/npadillacoreano/ryoi360/projects/reward...,/blue/npadillacoreano/ryoi360/projects/reward_...,"{'locations': [[[[nan] [nan]], [[nan] [nan]]...","[left_ear, right_ear, nose, tail_base, thorax,...","[[[[nan], [nan]], [[nan], [nan]], [[nan], [nan...",[1.1],{'1.1': 0},"{'1.1': [[[nan nan], [nan nan], [nan nan], [na..."
4,33400,68332,[1.1],1.1_1.4,1,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,1.1,[/blue/npadillacoreano/ryoi360/projects/reward...,/blue/npadillacoreano/ryoi360/projects/reward_...,{'locations': [[[[336.33197021] [400.23355103...,"[left_ear, right_ear, nose, tail_base, thorax,...","[[[[336.33197021], [400.23355103]], [[319.3523...",[1.1],{'1.1': 0},"{'1.1': [[[336.33197021 400.23355103], [319.35..."


## Getting the coordinates of the corners

In [75]:
START_STOP_FRAME_DF["sleap_path"].iloc[0]

'/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/id_corrected/20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1/20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1.1.1_subj.id_corrected.h5'

In [76]:
# Each corner file is the in the same folder and has the same basename of the pose tracking file 
START_STOP_FRAME_DF["corner_path"] = START_STOP_FRAME_DF["sleap_path"].apply(lambda x: x.replace("id_corrected.h5", "corner.h5").replace(".fixed", "").replace(".round_1", "").replace(".1_subj", "").replace(".2_subj", ""))


In [77]:
START_STOP_FRAME_DF["corner_path"].iloc[0]

'/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/id_corrected/20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1/20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1.1.corner.h5'

In [78]:
# Getting the indexes of each corner location
START_STOP_FRAME_DF["corner_parts"] = START_STOP_FRAME_DF["corner_path"].apply(lambda x: sleap.process_pose.get_node_names_from_sleap(x))

## TODO
- '20230625_112913_standard_comp_to_both_rewarded_D4_subj_1-1_and_1-4.1'
    - Look into why the corners have body parts instead

In [82]:
START_STOP_FRAME_DF["video_name"].iloc[25]

'20230625_112913_standard_comp_to_both_rewarded_D4_subj_1-1_and_1-4.1'

In [79]:
START_STOP_FRAME_DF["corner_parts"]

0     [box_top_left, box_top_right, reward_port, box...
1     [box_top_left, box_top_right, reward_port, box...
2     [box_top_left, box_top_right, reward_port, box...
3     [box_bottom_left, box_top_right, reward_port, ...
4     [box_bottom_left, box_top_right, reward_port, ...
5     [box_bottom_left, box_top_right, reward_port, ...
6     [box_bottom_left, box_top_right, reward_port, ...
7     [box_bottom_left, box_top_right, reward_port, ...
8     [box_bottom_left, box_top_right, reward_port, ...
9     [box_bottom_left, box_top_right, reward_port, ...
10    [box_bottom_left, box_top_right, reward_port, ...
11    [box_bottom_left, box_top_right, reward_port, ...
12    [box_bottom_left, box_top_right, reward_port, ...
13    [box_bottom_left, box_top_right, reward_port, ...
14    [box_bottom_left, box_top_right, reward_port, ...
15    [box_bottom_left, box_top_right, reward_port, ...
16    [box_bottom_left, box_top_right, reward_port, ...
17    [box_bottom_left, box_top_left, box_top_ri

In [80]:
START_STOP_FRAME_DF["corner_parts"].iloc[0]

['box_top_left',
 'box_top_right',
 'reward_port',
 'box_bottom_left',
 'box_bottom_right']

In [83]:
# TODO: Remove this once corner files are fixed
START_STOP_FRAME_DF = START_STOP_FRAME_DF[START_STOP_FRAME_DF["corner_parts"].apply(lambda x: "reward_port" in x)]

In [84]:
# Getting the coordinates of all the corners
START_STOP_FRAME_DF["corner_to_coordinate"] = START_STOP_FRAME_DF["corner_path"].apply(lambda x: sleap.process_pose.get_sleap_tracks_from_h5(x))

/scratch/local/24060335/ipykernel_2719052/277501212.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  START_STOP_FRAME_DF["corner_to_coordinate"] = START_STOP_FRAME_DF["corner_path"].apply(lambda x: sleap.process_pose.get_sleap_tracks_from_h5(x))


In [85]:
# Parsing out each corner and creating a dictionary of name to coordinates
START_STOP_FRAME_DF["corner_to_coordinate"] = START_STOP_FRAME_DF.apply(lambda x: {part: x["corner_to_coordinate"][:,index,:,:] for index, part in enumerate(x["corner_parts"])}, axis=1)

/scratch/local/24060335/ipykernel_2719052/129892389.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  START_STOP_FRAME_DF["corner_to_coordinate"] = START_STOP_FRAME_DF.apply(lambda x: {part: x["corner_to_coordinate"][:,index,:,:] for index, part in enumerate(x["corner_parts"])}, axis=1)


## TODO: Figure out why some corners are nan

In [88]:
START_STOP_FRAME_DF["corner_to_coordinate"]

0     {'box_top_left': [[[nan], [nan]], [[nan], [nan...
1     {'box_top_left': [[[nan], [nan]], [[nan], [nan...
2     {'box_top_left': [[[nan], [nan]], [[nan], [nan...
3     {'box_bottom_left': [[[234.69934175], [389.572...
4     {'box_bottom_left': [[[219.49634883], [382.936...
5     {'box_bottom_left': [[[219.49634883], [382.936...
6     {'box_bottom_left': [[[219.49634883], [382.936...
7     {'box_bottom_left': [[[234.47302231], [387.438...
8     {'box_bottom_left': [[[218.76658435], [382.320...
9     {'box_bottom_left': [[[218.76658435], [382.320...
10    {'box_bottom_left': [[[218.76658435], [382.320...
11    {'box_bottom_left': [[[234.49380639], [387.021...
12    {'box_bottom_left': [[[219.48526849], [382.698...
13    {'box_bottom_left': [[[220.39193729], [383.559...
14    {'box_bottom_left': [[[220.39193729], [383.559...
15    {'box_bottom_left': [[[220.39193729], [383.559...
16    {'box_bottom_left': [[[232.92760122], [389.100...
17    {'box_bottom_left': [[[215.66257348       

In [90]:
# Filtering out all the Nans because there's only one labeled frame
START_STOP_FRAME_DF["corner_to_coordinate"] = START_STOP_FRAME_DF.apply(lambda x: {k: v[~np.isnan(v)][:2] for k, v in x["corner_to_coordinate"].items()}, axis=1)

/scratch/local/24060335/ipykernel_2719052/2482341573.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  START_STOP_FRAME_DF["corner_to_coordinate"] = START_STOP_FRAME_DF.apply(lambda x: {k: v[~np.isnan(v)][:2] for k, v in x["corner_to_coordinate"].items()}, axis=1)


In [92]:
START_STOP_FRAME_DF["corner_to_coordinate"]

0     {'box_top_left': [215.09666220678088, 113.9234...
1     {'box_top_left': [215.09666220678088, 113.9234...
2     {'box_top_left': [215.09666220678088, 113.9234...
3     {'box_bottom_left': [234.69934174733294, 389.5...
4     {'box_bottom_left': [219.49634882813532, 382.9...
5     {'box_bottom_left': [219.49634882813532, 382.9...
6     {'box_bottom_left': [219.49634882813532, 382.9...
7     {'box_bottom_left': [234.47302230577947, 387.4...
8     {'box_bottom_left': [218.76658435012303, 382.3...
9     {'box_bottom_left': [218.76658435012303, 382.3...
10    {'box_bottom_left': [218.76658435012303, 382.3...
11    {'box_bottom_left': [234.49380639157778, 387.0...
12    {'box_bottom_left': [219.48526849018168, 382.6...
13    {'box_bottom_left': [220.39193729003364, 383.5...
14    {'box_bottom_left': [220.39193729003364, 383.5...
15    {'box_bottom_left': [220.39193729003364, 383.5...
16    {'box_bottom_left': [232.9276012199772, 389.10...
17    {'box_bottom_left': [215.66257348338468, 3

# Getting the distances between corners

- Getting the average width and height so that we can convert pixels to cm

In [93]:
# Using the x-coordinates for the width
START_STOP_FRAME_DF["bottom_width"] = START_STOP_FRAME_DF["corner_to_coordinate"].apply(lambda x: x["box_bottom_right"][0] - x["box_bottom_left"][0])
START_STOP_FRAME_DF["top_width"] = START_STOP_FRAME_DF["corner_to_coordinate"].apply(lambda x: x["box_top_right"][0] - x["box_top_left"][0])


/scratch/local/24060335/ipykernel_2719052/2243602241.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  START_STOP_FRAME_DF["bottom_width"] = START_STOP_FRAME_DF["corner_to_coordinate"].apply(lambda x: x["box_bottom_right"][0] - x["box_bottom_left"][0])
/scratch/local/24060335/ipykernel_2719052/2243602241.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  START_STOP_FRAME_DF["top_width"] = START_STOP_FRAME_DF["corner_to_coordinate"].apply(lambda x: x["box_top_right"][0] - x["box_top_left"][0])


In [94]:
# Using the y-coordinates for the height
START_STOP_FRAME_DF["right_height"] = START_STOP_FRAME_DF["corner_to_coordinate"].apply(lambda x: x["box_bottom_right"][1] - x["box_top_right"][1])
START_STOP_FRAME_DF["left_height"] = START_STOP_FRAME_DF["corner_to_coordinate"].apply(lambda x: x["box_bottom_left"][1] - x["box_top_left"][1])


/scratch/local/24060335/ipykernel_2719052/1007448330.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  START_STOP_FRAME_DF["right_height"] = START_STOP_FRAME_DF["corner_to_coordinate"].apply(lambda x: x["box_bottom_right"][1] - x["box_top_right"][1])
/scratch/local/24060335/ipykernel_2719052/1007448330.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  START_STOP_FRAME_DF["left_height"] = START_STOP_FRAME_DF["corner_to_coordinate"].apply(lambda x: x["box_bottom_left"][1] - x["box_top_left"][1])


In [95]:
# averaging the width and height by adding both sides and then getting the mean
START_STOP_FRAME_DF["average_height"] = START_STOP_FRAME_DF.apply(lambda row: (row["right_height"] + row["left_height"])/2, axis=1)
START_STOP_FRAME_DF["average_width"] = START_STOP_FRAME_DF.apply(lambda row: (row["bottom_width"] + row["top_width"])/2, axis=1)

/scratch/local/24060335/ipykernel_2719052/568356970.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  START_STOP_FRAME_DF["average_height"] = START_STOP_FRAME_DF.apply(lambda row: (row["right_height"] + row["left_height"])/2, axis=1)
/scratch/local/24060335/ipykernel_2719052/568356970.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  START_STOP_FRAME_DF["average_width"] = START_STOP_FRAME_DF.apply(lambda row: (row["bottom_width"] + row["top_width"])/2, axis=1)


## TODO: Figure out why average height is so low

In [96]:
START_STOP_FRAME_DF["average_height"]

0     266.715544
1     266.715544
2     266.715544
3     262.551593
4     263.215010
5     263.215010
6     263.215010
7     263.220692
8     260.889234
9     260.889234
10    260.889234
11    263.121313
12    262.412119
13    264.041942
14    264.041942
15    264.041942
16    261.262437
17    262.119650
18    262.119650
19    266.281781
20    266.281781
21    264.219046
22    264.219046
23      3.080119
24      3.080119
27    259.815529
28    259.815529
29    259.815529
30    259.815529
31    262.862896
32    262.862896
33    257.898587
34    257.898587
35    259.055487
36    259.055487
37    262.008238
38    262.008238
39    262.008238
40    262.008238
41    260.866315
42    260.866315
43    260.866315
44    260.866315
45    260.449185
46    260.449185
47    260.449185
48    260.449185
49    265.973648
50    265.973648
51    265.973648
52    265.973648
Name: average_height, dtype: float64

- Getthing the pixel to cm ratio

In [ ]:
START_STOP_FRAME_DF["width_ratio"] = MED_PC_WIDTH / START_STOP_FRAME_DF["average_width"]
START_STOP_FRAME_DF["height_ratio"] = MED_PC_HEIGHT / START_STOP_FRAME_DF["average_height"]

In [ ]:
START_STOP_FRAME_DF["height_ratio"]

In [ ]:
START_STOP_FRAME_DF["width_ratio"]

## Converting Pixels to cm

In [ ]:
START_STOP_FRAME_DF["subject_to_tracks"].iloc[0]["1.1"].shape

In [ ]:
START_STOP_FRAME_DF["in_video_subjects"] = START_STOP_FRAME_DF["in_video_subjects"].apply(lambda x: x.split("_"))

In [ ]:
START_STOP_FRAME_DF["subject_to_tracks"] = START_STOP_FRAME_DF.apply(lambda x: {k: v for k, v in x["subject_to_tracks"].items() if k in x["in_video_subjects"]}, axis=1)

- Converting the X-dimension

In [ ]:
START_STOP_FRAME_DF["subject_to_tracks"].head()

In [ ]:
START_STOP_FRAME_DF["rescaled_locations"] = START_STOP_FRAME_DF.apply(lambda x: {key: sleap.process_pose.fill_missing(sleap.process_pose.rescale_dimension_in_array(value, dimension=0, ratio=x["width_ratio"])) for key, value in x["subject_to_tracks"].items()}, axis=1)

- Converting the Y-dimension

In [ ]:
START_STOP_FRAME_DF["rescaled_locations"] = START_STOP_FRAME_DF.apply(lambda x: {key: sleap.process_pose.rescale_dimension_in_array(value, dimension=1, ratio=x["height_ratio"]) for key, value in x["rescaled_locations"].items()}, axis=1)

In [ ]:
START_STOP_FRAME_DF["corner_to_coordinate"]

In [ ]:
START_STOP_FRAME_DF.head()

In [ ]:
# Normalize dictionary column
normalized = pd.json_normalize(START_STOP_FRAME_DF["corner_to_coordinate"])

In [ ]:
normalized

In [ ]:


# Drop the original column and concat the normalized DataFrame
START_STOP_FRAME_DF = pd.concat([START_STOP_FRAME_DF.drop(["corner_to_coordinate"], axis=1), normalized], axis=1)


In [ ]:
START_STOP_FRAME_DF.head()

In [ ]:
START_STOP_FRAME_DF = START_STOP_FRAME_DF.dropna(subset=["reward_port"])

In [ ]:
for corner in START_STOP_FRAME_DF["corner_parts"].iloc[0]:
    START_STOP_FRAME_DF[corner] = START_STOP_FRAME_DF.apply(lambda x: [x[corner][0]*x["width_ratio"], x[corner][1]*x["height_ratio"]], axis=1)

In [ ]:
START_STOP_FRAME_DF

In [74]:
raise ValueError()

ValueError: 

## Looking over the tracks

In [ ]:
FILE_INDEX = 0

In [ ]:
LFP_AND_SLEAP_DF["sleap_path"].iloc[FILE_INDEX]

In [ ]:
LFP_AND_SLEAP_DF["rescaled_locations"]

In [ ]:
LFP_AND_SLEAP_DF["subject"]

In [ ]:
with h5py.File(LFP_AND_SLEAP_DF["sleap_path"].iloc[FILE_INDEX], "r") as f:
    dset_names = list(f.keys())
    current_subject = LFP_AND_SLEAP_DF["subject"].iloc[FILE_INDEX]
    locations = LFP_AND_SLEAP_DF["rescaled_locations"].iloc[FILE_INDEX][current_subject]
    node_names = [n.decode() for n in f["node_names"][:]]
    
print("===HDF5 datasets===")
print(dset_names)
print()

print("===locations data shape===")
print(locations.shape)
print()

print("===nodes===")
for i, name in enumerate(node_names):
    print(f"{i}: {name}")
print()

In [ ]:
thorax_loc = locations[:, THORAX_INDEX, :]

In [ ]:
fig, ax = plt.subplots()

plt.plot(thorax_loc[:,0],label='X-coordinates')
# Converting to negative so that we can see both x and y track
plt.plot(-1*thorax_loc[:,1], label='Y-coordinates')

plt.legend(loc="center right")
plt.title('Thorax locations')
plt.xlabel("Time in frames")
plt.ylabel("Coordinate Position")

In [ ]:
plt.figure(figsize=(7,7))
plt.plot(thorax_loc[:,0],thorax_loc[:,1])


plt.title('Thorax tracks')
plt.xlabel("X-Coordinates")
plt.ylabel("Y-Coordinates")


## Creating an individual column for each pose tracking

In [ ]:
LFP_AND_SLEAP_DF = LFP_AND_SLEAP_DF.dropna(subset="current_subject")

In [ ]:
# LFP_AND_SLEAP_DF["agent"] = LFP_AND_SLEAP_DF.apply(lambda x: list(set(x["all_subjects"]) - set(x["subject"]))[0], axis=1)

LFP_AND_SLEAP_DF["agent"] = LFP_AND_SLEAP_DF.apply(lambda x: list((set(x["in_video_subjects"]) - set([x["current_subject"]])))[0], axis=1)


In [ ]:
LFP_AND_SLEAP_DF

In [ ]:
LFP_AND_SLEAP_DF["subject_locations"] = LFP_AND_SLEAP_DF.apply(lambda x: x["rescaled_locations"][x["subject"]] , axis=1)

In [ ]:
LFP_AND_SLEAP_DF["agent_locations"] = LFP_AND_SLEAP_DF.apply(lambda x: x["rescaled_locations"].get(x["agent"], np.nan) , axis=1)

## Removing unnecessary columns

In [ ]:
LFP_AND_SLEAP_DF = LFP_AND_SLEAP_DF.drop(["sleap_glob", "subject_to_index", "subject_to_tracks", "corner_parts", "corner_to_coordinate", "bottom_width", "top_width", "right_height", "left_height", "average_height", "average_width", "width_ratio", "height_ratio", 'locations', 'current_subject', 'track_names', 'sleap_path', 'corner_path', 'all_sleap_data', 'rescaled_locations'], errors="ignore", axis=1)

In [ ]:
LFP_AND_SLEAP_DF.columns

In [ ]:
LFP_AND_SLEAP_DF.head()

In [ ]:
LFP_AND_SLEAP_DF["subject_locations"].apply(lambda x: x.shape)

# Calculate velocity

In [ ]:
LFP_AND_SLEAP_DF.columns

In [ ]:
LFP_AND_SLEAP_DF["body_parts"].apply(lambda x: x.index("thorax"))

In [ ]:
LFP_AND_SLEAP_DF["subject_thorax_velocity"] = LFP_AND_SLEAP_DF.apply(lambda x: compute_velocity(x["subject_locations"][:,x["body_parts"].index("thorax"),:], window_size=FRAME_RATE*3) * FRAME_RATE, axis=1)
LFP_AND_SLEAP_DF["subject_thorax_velocity"] = LFP_AND_SLEAP_DF["subject_thorax_velocity"].apply(lambda x: x.astype(np.float16) if x is not np.nan else np.nan)

In [ ]:
LFP_AND_SLEAP_DF["agent_thorax_velocity"] = LFP_AND_SLEAP_DF.apply(lambda x: compute_velocity(x["agent_locations"][:,x["body_parts"].index("thorax"),:], window_size=FRAME_RATE*3) * FRAME_RATE if x["agent_locations"] is not np.nan else np.nan, axis=1)
LFP_AND_SLEAP_DF["agent_thorax_velocity"] = LFP_AND_SLEAP_DF["agent_thorax_velocity"].apply(lambda x: x.astype(np.float16) if x is not np.nan else np.nan)


In [ ]:
LFP_AND_SLEAP_DF["subject_thorax_velocity"].iloc[0].shape

## Calculate distance to reward port

In [ ]:
LFP_AND_SLEAP_DF["subject_thorax_to_reward_port"] = LFP_AND_SLEAP_DF.apply(lambda x: np.linalg.norm(x["subject_locations"][:,x["body_parts"].index("thorax"),:] - x["reward_port"], axis=1),  axis=1)
LFP_AND_SLEAP_DF["subject_thorax_to_reward_port"] = LFP_AND_SLEAP_DF["subject_thorax_to_reward_port"].apply(lambda x: x.astype(np.float16) if x is not np.nan else np.nan)

In [ ]:
LFP_AND_SLEAP_DF["agent_thorax_to_reward_port"] = LFP_AND_SLEAP_DF.apply(lambda x: np.linalg.norm(x["agent_locations"][:,x["body_parts"].index("thorax"),:] - x["reward_port"], axis=1) if x["agent_locations"] is not np.nan else np.nan,  axis=1)
LFP_AND_SLEAP_DF["agent_thorax_to_reward_port"] = LFP_AND_SLEAP_DF["agent_thorax_to_reward_port"].apply(lambda x: x.astype(np.float16) if x is not np.nan else np.nan)

## Exporting

In [ ]:
LFP_AND_SLEAP_DF.columns

In [ ]:
LFP_AND_SLEAP_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [ ]:
LFP_AND_SLEAP_DF

# Velocity timestamps

In [ ]:
raise ValueError()

In [ ]:
MERGED_TRIAL_AND_SLEAP["full-recording_subject_thorax_velocity"] = MERGED_TRIAL_AND_SLEAP["full-recording_subject_thorax_location_all-frames"].apply(lambda x: compute_velocity(x, window_size=FRAME_RATE*3) * FRAME_RATE)

MERGED_TRIAL_AND_SLEAP["full-recording_agent_thorax_velocity"] = MERGED_TRIAL_AND_SLEAP["full-recording_agent_thorax_location_all-frames"].apply(lambda x: compute_velocity(x, window_size=FRAME_RATE*3) * FRAME_RATE if x is not np.nan else np.nan)

In [ ]:
MERGED_TRIAL_AND_SLEAP["full-recording_subject_thorax_velocity"] = MERGED_TRIAL_AND_SLEAP["full-recording_subject_thorax_location_all-frames"].apply(lambda x: compute_velocity(x, window_size=FRAME_RATE*3) * FRAME_RATE)

MERGED_TRIAL_AND_SLEAP["full-recording_agent_thorax_velocity"] = MERGED_TRIAL_AND_SLEAP["full-recording_agent_thorax_location_all-frames"].apply(lambda x: compute_velocity(x, window_size=FRAME_RATE*3) * FRAME_RATE if x is not np.nan else np.nan)

# OLD CODE BELOW

# Adding the start/stop frame information

In [ ]:
subject_locations.head()

- Getting relevant metadata for each video

In [ ]:
# Getting all the rows that have two subjects
subject_locations["tracked_subject"] = subject_locations["tracked_subject"].apply(lambda x: str(x).split("_"))
subject_locations = subject_locations[subject_locations["tracked_subject"].apply(lambda x: len(x) == 2)]

In [ ]:
# Getting the sleap filename from file path
subject_locations["sleap_filename"] = subject_locations["file_path"].apply(lambda x: os.path.basename(x))

In [ ]:
# Getting the sleap fileroot from the sleap filename
subject_locations["sleap_fileroot"] = subject_locations["sleap_filename"].apply(lambda x: ".".join(x.split(".")[0:2]))

In [ ]:
# Combining the start and stop frame columns into a tuple
subject_locations["start_stop_frame"] = subject_locations.apply(lambda x: (int(x["start_frame"]), int(x["stop_frame"])), axis=1)
subject_locations = subject_locations.drop(columns=["start_frame", "stop_frame"], errors="ignore")

- Merging the dataframes based on shared SLEAP file basename

In [ ]:
LFP_AND_SLEAP_DF = pd.merge(left=LFP_AND_SLEAP_DF, right=subject_locations, left_on="video_name", right_on="sleap_fileroot", how="left")

- Converting the start/stop frames into timestamps

In [ ]:
LFP_AND_SLEAP_DF["start_stop_timestamps"] = LFP_AND_SLEAP_DF.apply(lambda x: (x["video_timestamps"][x["start_stop_frame"][0]], x["video_timestamps"][x["start_stop_frame"][1]]), axis=1)

## Going from frame information to ephys

# Filtering for parts of the video

In [ ]:
for trace_col in [col for col in LFP_AND_SLEAP_DF.columns if "lfp_trace" in col]:
    print(trace_col)
    brain_region = trace_col.split("_")[0]
    LFP_AND_SLEAP_DF["filtered_{}_trace".format(brain_region)] = LFP_AND_SLEAP_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(x["start_stop_timestamps"][0], x["start_stop_timestamps"][1], x["lfp_timestamps"], x[trace_col])[1], axis=1)
LFP_AND_SLEAP_DF["filtered_lfp_timestamps"] = LFP_AND_SLEAP_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(x["start_stop_timestamps"][0], x["start_stop_timestamps"][1], x["lfp_timestamps"], x["lfp_timestamps"])[0], axis=1)

In [ ]:
LFP_AND_SLEAP_DF = LFP_AND_SLEAP_DF.drop(columns=[col for col in LFP_AND_SLEAP_DF.columns if "lfp_trace" in col], errors="ignore")
LFP_AND_SLEAP_DF = LFP_AND_SLEAP_DF.drop(columns=["lfp_timestamps"], errors="ignore")

In [ ]:
LFP_AND_SLEAP_DF.head()

In [ ]:
LFP_AND_SLEAP_DF["]

In [ ]:
LFP_AND_SLEAP_DF["video_timestamps"].iloc[0]

In [ ]:
LFP_AND_SLEAP_DF["video_timestamps"].apply(lambda x: x.shape)

In [ ]:
LFP_AND_SLEAP_DF["subject_locations"].apply(lambda x: x.shape)

In [ ]:
LFP_AND_SLEAP_DF["recording"].iloc[5]

In [ ]:
LFP_AND_SLEAP_DF["filtered_subject_locations"] = LFP_AND_SLEAP_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(x["start_stop_timestamps"][0], x["start_stop_timestamps"][1], x["video_timestamps"], x["subject_locations"])[1], axis=1)
LFP_AND_SLEAP_DF["filtered_agent_locations"] = LFP_AND_SLEAP_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(x["start_stop_timestamps"][0], x["start_stop_timestamps"][1], x["video_timestamps"], x["agent_locations"])[1], axis=1)
LFP_AND_SLEAP_DF["filtered_video_timestamps"] = LFP_AND_SLEAP_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(x["start_stop_timestamps"][0], x["start_stop_timestamps"][1], x["video_timestamps"], x["video_timestamps"])[0], axis=1)

In [ ]:
LFP_AND_SLEAP_DF = LFP_AND_SLEAP_DF.drop(columns=["video_timestamps", "subject_locations", "agent_locations"], errors="ignore")

In [ ]:
# Sorting column names for easier reading
sorted_columns = sorted(LFP_AND_SLEAP_DF.columns, key=lambda x: x.split("_")[-1])

In [ ]:
LFP_AND_SLEAP_DF = LFP_AND_SLEAP_DF[sorted_columns].copy()

In [ ]:
LFP_AND_SLEAP_DF.columns

In [ ]:
LFP_AND_SLEAP_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [ ]:
LFP_AND_SLEAP_DF

In [ ]:
raise ValueError()

In [ ]:
def find_nearest_timestamp_indices(timestamps, other_timestamps, start_index=0, stop_index=1):
    """
    Converts the start and stop indices of one data stream to timestamps, and then finds the nearest start and stop 
    timestamps in another data stream.

    Parameters:
    - timestamps (list[int or float]): The list of timestamps in the first data stream.
    - other_timestamps (list[int or float]): The list of timestamps in the other data stream.
    - start_index (int, optional): The start index in the first data stream. Defaults to 0.
    - stop_index (int, optional): The stop index in the first data stream. Defaults to 1.

    Returns:
    - tuple: The indices of the nearest start and stop timestamps in the other data stream.
    """
    # Convert start and stop indices to timestamps
    start_timestamp = timestamps[start_index]
    stop_timestamp = timestamps[stop_index]

    # Find nearest start and stop timestamps in other data stream
    nearest_start_index = utilities.helper.find_nearest_index(other_timestamps, start_timestamp)
    nearest_stop_index = utilities.helper.find_nearest_index(other_timestamps, stop_timestamp) 

    return nearest_start_index, nearest_stop_index

In [ ]:
raise ValueError()

- Getting the names of each subject

In [ ]:
LFP_AND_SLEAP_DF["video_name"].unique()

In [ ]:
glob.glob(SLEAP_DIR+ "/*/*id_corrected*.h5")[:10]

In [ ]:
SLEAP_DIR

In [ ]:
START_STOP_FRAME_DF

In [ ]:
# LFP_AND_SLEAP_DF["video_path"] = VIDEO_TO_FRAME_AND_SUBJECT_DF["video_name"].apply(lambda x: os.path.join(SLEAP_DIR, "*", x + "*.h5"))
VIDEO_TO_FRAME_AND_SUBJECT_DF["sleap_glob"] = VIDEO_TO_FRAME_AND_SUBJECT_DF["video_name"].apply(lambda x: glob.glob(os.path.join(SLEAP_DIR, "*", x + "*id_corrected*.h5")))
# VIDEO_TO_FRAME_AND_SUBJECT_DF["sleap_glob"] = VIDEO_TO_FRAME_AND_SUBJECT_DF["video_name"].apply(lambda x: os.path.join(SLEAP_DIR, "*", x + "*2_subj*.h5"))


In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF = VIDEO_TO_FRAME_AND_SUBJECT_DF[VIDEO_TO_FRAME_AND_SUBJECT_DF['sleap_glob'].apply(lambda x: len(x) >= 1)]
VIDEO_TO_FRAME_AND_SUBJECT_DF = VIDEO_TO_FRAME_AND_SUBJECT_DF.reset_index(drop=True)




In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF["sleap_glob"].iloc[0]

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF.columns

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF

# OLD merging code

## Putting together LFP and video start/stop

In [ ]:
START_STOP_FRAME_DF["video_name"].unique()[:5]

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF["video_name"].unique()[:5]

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF["current_subject"].unique()

In [ ]:
START_STOP_FRAME_DF["current_subject"].unique()

In [ ]:
LFP_AND_SLEAP_DF = pd.merge(VIDEO_TO_FRAME_AND_SUBJECT_DF, START_STOP_FRAME_DF, on=["video_name", "current_subject"], how="inner")

In [ ]:
LFP_AND_SLEAP_DF.head()

In [ ]:
LFP_AND_SLEAP_DF["video_timestamps"].apply(lambda x: x.shape).head()